# MNIST Model From "Scratch" -- Attempt 2

Goal: To reproduce the second half of Chapter 4 from the Fast.ai book with as little referencing the material as possible. Looking up things in the documentation is allowed.

In [1]:
from fastai.data.external import untar_data, URLs
from pathlib import Path

data = untar_data(URLs.MNIST_SAMPLE)
Path.Base_Path = data

In [2]:
from PIL import Image

image = Image.open((data/'valid'/'3').ls()[0])
image

In [3]:
import torch

# These imports are sloppy...
import numpy as np
from numpy import *

valid_3_tens = [torch.as_tensor(array(Image.open(im))) for im in (data/'valid'/'3').ls()]
valid_7_tens = [torch.as_tensor(array(Image.open(im))) for im in (data/'valid'/'7').ls()]
valid_3_tens[0].shape, valid_7_tens[0].shape

(torch.Size([28, 28]), torch.Size([28, 28]))

In [4]:
valid_3_tens_stacked = torch.stack(valid_3_tens)
valid_7_tens_stacked = torch.stack(valid_7_tens)
valid_3_tens_stacked.shape, valid_7_tens_stacked.shape

(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [5]:
valid_3s_view = valid_3_tens_stacked.view(-1, 28*28).to(torch.float64)/255
valid_7s_view = valid_7_tens_stacked.view(-1, 28*28).to(torch.float64)/255
valid_3s_view.shape, valid_7s_view.shape

(torch.Size([1010, 784]), torch.Size([1028, 784]))

In [6]:
x = torch.cat([valid_3s_view, valid_7s_view])
x.shape

torch.Size([2038, 784])

In [7]:
y = torch.cat([torch.as_tensor([1] * len(valid_3s_view)), torch.as_tensor([0] * len(valid_7s_view))]).unsqueeze(-1)
y.shape, y[0:4]

(torch.Size([2038, 1]),
 tensor([[1],
         [1],
         [1],
         [1]]))

In [8]:
# I'm not totally sure this bit is right. I think I want values that are between 0 and 1. We'll see...

def init_param(shape, std=1.0): return (torch.as_tensor(np.random.randn(shape)) + std).requires_grad_()

weights = torch.as_tensor(np.random.randn(784)).requires_grad_()
bias = torch.as_tensor(np.random.randn(1)).requires_grad_()
weights.shape, bias.shape
weights[0:100]

tensor([ 0.4544, -0.8108, -1.9538, -1.3584, -1.3897,  0.0122, -0.1243,  0.0540,
        -2.3786, -0.2573,  0.1538, -0.2044,  1.2569, -0.0211,  2.1899, -0.3923,
        -0.5945, -1.0011,  0.5230,  0.6760,  1.6610,  2.2277,  0.7094,  1.6383,
         0.5006, -0.1470,  0.5744,  0.7620,  0.1112,  0.0907, -0.4662,  0.1981,
        -0.2238, -0.0942,  1.0714,  0.2125,  0.2238, -1.2645, -0.3131,  0.6188,
        -0.5867, -0.0473,  1.3559, -0.9897, -0.1669, -1.0384,  0.2056,  0.3999,
        -1.5592, -1.7413,  0.2792,  0.2361,  1.1848,  0.4932, -0.2027, -0.7021,
         0.7435, -1.0416, -0.2277,  1.0773,  0.4993, -1.3881, -0.3282,  0.6901,
         1.1101,  0.2161, -0.7514, -0.2215, -1.1100, -0.1379, -0.7076, -1.2148,
        -0.1597,  1.3133,  1.1880, -0.2805, -0.7836,  0.8343, -1.5228,  0.1492,
        -1.1477,  0.4082,  0.1313,  0.6022, -1.5428,  0.7545, -0.2222, -0.3892,
        -0.3660, -0.5062, -0.6368,  1.3330, -1.7704, -0.0309,  1.4297, -1.1189,
         0.9362,  0.8946,  0.9181, -0.28

In [9]:
def linear1(x): return x@weights + bias

In [34]:
def mnist_loss(preds, targets): return torch.where(targets==1, 1-preds, preds).mean() # initially forgot the mean, didn't catch it at first because my initial sample was only one image.

In [11]:
# loss test

preds = torch.as_tensor([0.4, 0.1, 0.9])
targets = torch.as_tensor([0, 0, 1])
mnist_loss(preds, targets)

tensor([0.4000, 0.1000, 0.1000])

In [12]:
# One round of SGD for a 1-image batch, "manually"

pred_sample = linear1(x[0])
pred_sample

tensor([-2.6534], dtype=torch.float64, grad_fn=<AddBackward0>)

In [13]:
# Damn, I really messed this up somewhere...

# Not sure what that 2.4... is supposed to signify...

# Maybe I just to sigmoid that number, then compare it to the 0 or 1 to give the loss.

In [14]:
pred_sample.sigmoid_()

tensor([0.0658], dtype=torch.float64, grad_fn=<SigmoidBackward0>)

In [15]:
loss = mnist_loss(pred_sample, y[0])
loss

tensor([0.9342], dtype=torch.float64, grad_fn=<WhereBackward0>)

In [16]:
loss.backward()
weights.grad.shape

torch.Size([784])

In [17]:
weights.grad.shape, bias.grad.shape

(torch.Size([784]), torch.Size([1]))

In [18]:
def step_once(params, lr=0.001):
    for param in params:
        param.data -= param.grad * lr
        param.grad = None

In [19]:
step_once([weights, bias])

In [20]:
weights.grad

In [21]:
# I don't understand how the weights aren't leaf tensors...

# Ohhhh, it has to do with the fact that I was pulling a single weight off before updating it. I need to update the entire tensor at once...

# ...Yep! That did it.

In [22]:
pred_sample_2 = linear1(x[0]).sigmoid()
print(pred_sample_2)
loss = mnist_loss(pred_sample_2, y[0])
print(loss)
loss.backward()


tensor([0.0662], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
tensor([0.9338], dtype=torch.float64, grad_fn=<WhereBackward0>)


In [23]:
loss.shape

torch.Size([1])

In [29]:
# Zip up Xs and Ys, get into dataloader, train for an epoch

training_data = list(zip(x, y))
training_data[0][0].shape

torch.Size([784])

In [33]:
from fastai.data.load import DataLoader

dl = DataLoader(dataset=training_data, batch_size=250)
dl

### Wow, I was about to train on the validation data...hahaha.

In [35]:
def train_one_epoch(dataloader, model):
    for xb, yb in dataloader:
        preds = model(xb).sigmoid() # probably should have put the sigmoid in the model
        loss = mnist_loss(preds, yb)
        print('loss: ', loss)
        loss.backward()
        step_once([weights, bias])

### Stopping here, because it's almost bed time.

Recap:
- Got through a round of gradient updates! Solid progress.
- Didn't look at the reference notebook once!
- Got the data into tensors really quickly.
- Missed:
    - Was about to train on the validation set.
    - Tried to update the weight gradients one-by-one, instead of doing the whole tensor at once.
    - Forgot to normalize the image data again, then ran into some weirdness with float32 vs float64. Had to explicitly cast, not sure why that happened this time.
    - Forgot to take the mean of the losses (after the torch.where call).